In [10]:
import pickle
import pymorphy3
import re
import tkinter as tk
from tkinter import Frame, Label, Button, Toplevel
import joblib

with open('ukrainian_stopwords.pkl', 'rb') as f:
    ukrainian_stopwords = pickle.load(f)

def preprocess(text):
    text = text.strip()
    text = re.sub(r'[^\w\s]', ' ', text) 
    text = text.lower()
    text = re.sub(r'\s+', ' ', text) 
    tokens = text.split()
    tokens = [word for word in tokens if len(word) > 1]
    tokens = [word for word in tokens if word not in ukrainian_stopwords]
    morph = pymorphy3.MorphAnalyzer(lang='uk')
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    tokens = [word for word in tokens if morph.parse(word)[0].tag.POS in {'NOUN', 'VERB', 'ADJ', 'ADV', 'ADJF'}]
    text = ' '.join(tokens)
    return text

with open('other/samples.txt', 'r') as file:
    content = file.read()

content = content.replace('"', '')
samples = [word.strip() for word in content.split('\n')]

reviews = []
l_s = len(samples)
for i in range(l_s):
    if i < 5:
        sent = "Негативний"
    elif i >= 5 and i < 10:
        sent = "Нейтральний"
    else: 
        sent = "Позитивний"
    reviews.append({"title": samples[i][:30]+'...', "content":samples[i], "sentiment":sent})
    


# Load the model and vectorizer
model = joblib.load("models/linear_svc.joblib")
vectorizer = joblib.load("models/vectorizer.joblib")

# Sentiment color mapping
sentiment_colors = {
    "Позитивний": "#4CAF50",  # Green
    "Нейтральний": "#FFC107",  # Amber
    "Негативний": "#F44336",  # Red
}

def analyze_text():
    """Analyze the sentiment of the input text."""
    text = text_input.get("1.0", "end-1c").strip()
    if not text:
        tk.messagebox.showwarning("Input Error", "Please enter some text for analysis.")
        return
    text_vectorized = vectorizer.transform([preprocess(text)])
    sentiment = model.predict(text_vectorized)[0]
    sentiment_map = {0: "Негативний", 1: "Нейтральний", 2: "Позитивний"}
    result_label.config(text=f"Настрій: {sentiment_map[sentiment]}")

def clear_text():
    """Clear the input text and result label."""
    text_input.delete("1.0", "end")
    result_label.config(text="")

def load_review():
    """Open the Load Review window and manage review selection."""
    def populate_review_list():
        """Populate the list with reviews."""
        for widget in review_list_frame.winfo_children():
            widget.destroy()  # Clear previous entries

        for idx, review in enumerate(reviews):
            # Create a frame for each review
            item_frame = Frame(review_list_frame, bg="white", padx=10, pady=5, relief="ridge", bd=1)
            item_frame.pack(fill="x", pady=5)

            def select_review(event, i=idx, frame=item_frame):
                """Handle review selection and styling."""
                nonlocal selected_review_idx
                if selected_review_idx != -1:  # Deselect the previous frame
                    previous_frame = review_list_frame.winfo_children()[selected_review_idx]
                    previous_frame.configure(bg="white")
                # Update the new selection
                selected_review_idx = i
                frame.configure(bg="#FFC0CB")  # Highlight the selected frame

            # Bind click event to select review
            item_frame.bind("<Button-1>", select_review)

            # Title
            title_label = Label(item_frame, text=review["title"], font=("Arial", 12, "bold"), bg="white", anchor="w")
            title_label.pack(side="left", expand=True, fill="x", padx=5)
            title_label.bind("<Button-1>", select_review)

            # Sentiment
            sentiment_label = Label(
                item_frame,
                text=review["sentiment"],
                font=("Arial", 12, "bold"),
                bg=sentiment_colors[review["sentiment"]],
                fg="white",
                padx=10,
                pady=5,
                relief="solid"
            )
            sentiment_label.pack(side="right", padx=5)
            sentiment_label.bind("<Button-1>", select_review)

    def load_selected_review():
        """Transfer selected review to the main window text field.""" 
        if selected_review_idx == -1:
            tk.messagebox.showwarning("Selection Error", "Please select a review to load.")
            return
        text_input.delete("1.0", "end")
        text_input.insert("1.0", reviews[selected_review_idx]["content"])
        review_window.destroy()

    def resize_canvas(event):
        """Adjust the canvas width to match the container frame."""
        canvas_width = event.width
        canvas.itemconfig(scrollable_window, width=canvas_width)

    # Initialize variables
    selected_review_idx = -1

    # Create the Load Window
    review_window = Toplevel(window)
    review_window.title("Приклади рев'ю")
    review_window.geometry("600x500")
    review_window.configure(bg="#f5f5f5")
    review_window.resizable(False, False)

    # Title
    title_label = Label(
        review_window, text="Оберіть рев'ю", font=("Arial", 14, "bold"), bg="#f5f5f5", fg="#333"
    )
    title_label.pack(pady=10)

    # Create a frame with scrollbar for reviews
    container_frame = Frame(review_window, bg="#f5f5f5")
    container_frame.pack(fill="both", expand=True, padx=10, pady=10)

    canvas = tk.Canvas(container_frame, bg="#f5f5f5", highlightthickness=0)
    scrollbar = tk.Scrollbar(container_frame, orient="vertical", command=canvas.yview)
    scrollable_frame = Frame(canvas, bg="#f5f5f5", padx=10)

    scrollable_window = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

    # Bind resize event to dynamically adjust canvas width
    container_frame.bind("<Configure>", resize_canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    review_list_frame = scrollable_frame

    # Populate initial reviews
    populate_review_list()

    # Load Button
    load_button = Button(
        review_window,
        text="Завантажити обране рев'ю",
        command=load_selected_review,
        font=("Arial", 12, "bold"),
        bg="#4CAF50",
        fg="white",
        activebackground="#45a049",
        relief="flat",
        padx=10,
        pady=5
    )
    load_button.pack(pady=10)

# Main Window Setup
window = tk.Tk()
window.title("Модуль сентиментального аналізу тексту")
window.geometry("550x520")
window.configure(bg="#f5f5f5")

# Title label
title_label = Label(window, text="Модуль сентиментального аналізу тексту", font=("Arial", 18, "bold"), bg="#f5f5f5", fg="#333")
title_label.pack(pady=10)

# Instruction label
instruction_label = Label(window, text="Введіть ваше рев'ю нижче:", font=("Arial", 12), bg="#f5f5f5", fg="#555")
instruction_label.pack()

# Text input field
text_input = tk.Text(window, height=14, width=50, font=("Arial", 12), bd=2, relief="solid", padx=10, pady=10)
text_input.pack(pady=10)

# Button row
button_frame = Frame(window, bg="#f5f5f5")
button_frame.pack(pady=10)


# Load Review button
load_button = Button(
    button_frame,
    text="Завантажити",
    command=load_review,
    font=("Arial", 12, "bold"),
    bg="#2196F3",
    fg="white",
    activebackground="#1E88E5",
    relief="flat",
    padx=10,
    pady=5
)
load_button.pack(side="left", padx=5)

# Analyze button
analyze_button = Button(
    button_frame,
    text="Аналізувати",
    command=analyze_text,
    font=("Arial", 12, "bold"),
    bg="#4CAF50",
    fg="white",
    activebackground="#45a049",
    relief="flat",
    padx=10,
    pady=5
)
analyze_button.pack(side="left", padx=5)

# Clear button
clear_button = Button(
    button_frame,
    text="Очистити",
    command=clear_text,
    font=("Arial", 12, "bold"),
    bg="#F44336",
    fg="white",
    activebackground="#E53935",
    relief="flat",
    padx=10,
    pady=5
)
clear_button.pack(side="left", padx=5)

# Result label
result_label = Label(window, text="", font=("Arial", 16, "bold"), bg="#f5f5f5", fg="#333", wraplength=450, justify="center")
result_label.pack(pady=12)

# Footer label
footer_label = Label(window, text="Зроблено Пархоменко Едуардом, всі права захищені", font=("Arial", 10, "italic"), bg="#f5f5f5", fg="#888")
footer_label.pack(side="bottom", pady=5)

# Run the application
window.mainloop()


In [2]:
def preprocess(text):
    text = text.strip()
    text = re.sub(r'[^\w\s]', ' ', text) 
    text = text.lower()
    text = re.sub(r'\s+', ' ', text) 
    tokens = text.split()
    tokens = [word for word in tokens if len(word) > 1]
    tokens = [word for word in tokens if word not in ukrainian_stopwords]
    morph = pymorphy3.MorphAnalyzer(lang='uk')
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    tokens = [word for word in tokens if morph.parse(word)[0].tag.POS in {'NOUN', 'VERB', 'ADJ', 'ADV', 'ADJF'}]
    text = ' '.join(tokens)
    return text

In [3]:
with open('other/samples.txt', 'r') as file:
    content = file.read()

content = content.replace('"', '')
samples = [word.strip() for word in content.split('\n')]
samples

["'Я лише хочу, щоб для цієї індички був рейтинг нижче 1.Що я можу сказати: погане редагування (о, і вам не потрібно бути експертом, щоб зловити це), страшна акторська майстерність та діалог, смішні сцени боротьби, дурний розвиток персонажів і далі. Сігал, мабуть, не вистачає виплат аліментів чи щось, щоб скласти цей безлад і розпродаж його для глядачів.Більшість героїв дій стають кращими з досвідом, але найдавніші твори Seagal настільки перевершують це. Замість того, щоб бути напівпристойним персонажем, як військово-морська печатка, або оперативник ЦРУ чи поліцейського, він грає в археолога, який одягається, як він прослуховує, що він не є в Матриці або Джона Шафта.Виглядає абсолютно смішно на копаннях у пустелі Китаю.Вони намагаються пояснити заднім числом, де цей професор отримав свої вищі бойові навички кунг-фу, але це працює так само погано, як і їхня спроба пояснити, як засуджений отримує доктор з археології, перебуваючи у в\\'язниці (без будь-яких польових робіт, звичайно). Тоді

In [4]:
len(samples)

15

In [5]:
reviews = []
l_s = len(samples)
for i in range(l_s):
    if i <= l_s / 3:
        sent = "Негативний"
    elif i > l_s / 3 and i <= l_s / 3 * 2:
        sent = "Нейтральний"
    else: 
        sent = "Позитивний"
    reviews.append({"title": samples[i][:30]+'...', "content":samples[i], "sentiment":sent})

In [6]:
import tkinter as tk
from tkinter import Frame, Label, Button, Toplevel, messagebox
import joblib

# Load the model and vectorizer
model = joblib.load("models/linear_svc.joblib")
vectorizer = joblib.load("models/vectorizer.joblib")

# Sentiment color mapping
sentiment_colors = {
    "Позитивний": "#4CAF50",  # Green
    "Нейтральний": "#FFC107",  # Amber
    "Негативний": "#F44336",  # Red
}

def analyze_text():
    """Analyze the sentiment of the input text."""
    text = text_input.get("1.0", "end-1c").strip()
    if not text:
        tk.messagebox.showwarning("Input Error", "Please enter some text for analysis.")
        return
    text_vectorized = vectorizer.transform([preprocess(text)])
    sentiment = model.predict(text_vectorized)[0]
    sentiment_map = {0: "Негативний", 1: "Нейтральний", 2: "Позитивний"}
    result_label.config(text=f"Настрій: {sentiment_map[sentiment]}")

def clear_text():
    """Clear the input text and result label."""
    text_input.delete("1.0", "end")
    result_label.config(text="")

def load_review():
    """Open the Load Review window and manage review selection."""
    def populate_review_list():
        """Populate the list with reviews."""
        for widget in review_list_frame.winfo_children():
            widget.destroy()  # Clear previous entries

        for idx, review in enumerate(reviews):
            # Create a frame for each review
            item_frame = Frame(review_list_frame, bg="white", padx=10, pady=5, relief="ridge", bd=1)
            item_frame.pack(fill="x", pady=5)

            def select_review(event, i=idx, frame=item_frame):
                """Handle review selection and styling."""
                nonlocal selected_review_idx
                if selected_review_idx != -1:  # Deselect the previous frame
                    previous_frame = review_list_frame.winfo_children()[selected_review_idx]
                    previous_frame.configure(bg="white")
                # Update the new selection
                selected_review_idx = i
                frame.configure(bg="#FFC0CB")  # Highlight the selected frame

            # Bind click event to select review
            item_frame.bind("<Button-1>", select_review)

            # Title
            title_label = Label(item_frame, text=review["title"], font=("Arial", 12, "bold"), bg="white", anchor="w")
            title_label.pack(side="left", expand=True, fill="x", padx=5)
            title_label.bind("<Button-1>", select_review)

            # Sentiment
            sentiment_label = Label(
                item_frame,
                text=review["sentiment"],
                font=("Arial", 12, "bold"),
                bg=sentiment_colors[review["sentiment"]],
                fg="white",
                padx=10,
                pady=5,
                relief="solid"
            )
            sentiment_label.pack(side="right", padx=5)
            sentiment_label.bind("<Button-1>", select_review)

    def load_selected_review():
        """Transfer selected review to the main window text field.""" 
        if selected_review_idx == -1:
            tk.messagebox.showwarning("Selection Error", "Please select a review to load.")
            return
        text_input.delete("1.0", "end")
        text_input.insert("1.0", reviews[selected_review_idx]["content"])
        review_window.destroy()

    def resize_canvas(event):
        """Adjust the canvas width to match the container frame."""
        canvas_width = event.width
        canvas.itemconfig(scrollable_window, width=canvas_width)

    # Initialize variables
    selected_review_idx = -1

    # Create the Load Window
    review_window = Toplevel(window)
    review_window.title("Приклади рев'ю")
    review_window.geometry("600x500")
    review_window.configure(bg="#f5f5f5")
    review_window.resizable(False, False)

    # Title
    title_label = Label(
        review_window, text="Оберіть рев'ю", font=("Arial", 14, "bold"), bg="#f5f5f5", fg="#333"
    )
    title_label.pack(pady=10)

    # Create a frame with scrollbar for reviews
    container_frame = Frame(review_window, bg="#f5f5f5")
    container_frame.pack(fill="both", expand=True, padx=10, pady=10)

    canvas = tk.Canvas(container_frame, bg="#f5f5f5", highlightthickness=0)
    scrollbar = tk.Scrollbar(container_frame, orient="vertical", command=canvas.yview)
    scrollable_frame = Frame(canvas, bg="#f5f5f5", padx=10)

    scrollable_window = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

    # Bind resize event to dynamically adjust canvas width
    container_frame.bind("<Configure>", resize_canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    review_list_frame = scrollable_frame

    # Populate initial reviews
    populate_review_list()

    # Load Button
    load_button = Button(
        review_window,
        text="Завантажити обране рев'ю",
        command=load_selected_review,
        font=("Arial", 12, "bold"),
        bg="#4CAF50",
        fg="white",
        activebackground="#45a049",
        relief="flat",
        padx=10,
        pady=5
    )
    load_button.pack(pady=10)

# Main Window Setup
window = tk.Tk()
window.title("Модуль сентиментального аналізу тексту")
window.geometry("550x520")
window.configure(bg="#f5f5f5")

# Title label
title_label = Label(window, text="Модуль сентиментального аналізу тексту", font=("Arial", 18, "bold"), bg="#f5f5f5", fg="#333")
title_label.pack(pady=10)

# Instruction label
instruction_label = Label(window, text="Введіть ваше рев'ю нижче:", font=("Arial", 12), bg="#f5f5f5", fg="#555")
instruction_label.pack()

# Text input field
text_input = tk.Text(window, height=14, width=50, font=("Arial", 12), bd=2, relief="solid", padx=10, pady=10)
text_input.pack(pady=10)

# Button row
button_frame = Frame(window, bg="#f5f5f5")
button_frame.pack(pady=10)


# Load Review button
load_button = Button(
    button_frame,
    text="Завантажити",
    command=load_review,
    font=("Arial", 12, "bold"),
    bg="#2196F3",
    fg="white",
    activebackground="#1E88E5",
    relief="flat",
    padx=10,
    pady=5
)
load_button.pack(side="left", padx=5)

# Analyze button
analyze_button = Button(
    button_frame,
    text="Аналізувати",
    command=analyze_text,
    font=("Arial", 12, "bold"),
    bg="#4CAF50",
    fg="white",
    activebackground="#45a049",
    relief="flat",
    padx=10,
    pady=5
)
analyze_button.pack(side="left", padx=5)

# Clear button
clear_button = Button(
    button_frame,
    text="Очистити",
    command=clear_text,
    font=("Arial", 12, "bold"),
    bg="#F44336",
    fg="white",
    activebackground="#E53935",
    relief="flat",
    padx=10,
    pady=5
)
clear_button.pack(side="left", padx=5)

# Result label
result_label = Label(window, text="", font=("Arial", 16, "bold"), bg="#f5f5f5", fg="#333", wraplength=450, justify="center")
result_label.pack(pady=12)

# Footer label
footer_label = Label(window, text="Зроблено Пархоменко Едуардом, всі права захищені", font=("Arial", 10, "italic"), bg="#f5f5f5", fg="#888")
footer_label.pack(side="bottom", pady=5)

# Run the application
window.mainloop()
